In [1]:
from tg.grammar_ru import *
import os
import pandas as pd
import numpy as np
from yo_fluq_ds import *


rucoco_dir = Loc.data_cache_path/'rucoco'

In [2]:
from jordan_lib.preprocessing import translate_frame_and_build_dict

def get_frames():
    bound = 1500 #2589
    def iter_frames():
        c = 0
        for entry in os.scandir(rucoco_dir/'frames'):
            yield pd.read_csv(entry).drop(columns=['Unnamed: 0'])
            c += 1
            if c == bound:
                break
    return Queryable(iter_frames(), bound)
    
vocab = {}
text_result = []
caps = {}
get_frames().feed(fluq.with_progress_bar()).foreach(lambda z: translate_frame_and_build_dict(z, vocab, text_result, caps))

  0%|          | 0/1500 [00:00<?, ?it/s]

In [3]:
get_frames().first()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,category
0,0,0,0,0,1,Компромисс,ru,10,0.0
1,1,0,1,0,1,между,ru,5,0.0
2,2,0,2,0,1,депутатами,ru,10,1.0
3,3,0,3,0,1,и,ru,1,0.0
4,4,0,4,0,1,правительством,ru,14,2.0
...,...,...,...,...,...,...,...,...,...
363,359,15,24,12,1,был,ru,3,0.0
364,360,15,25,12,1,внесен,ru,6,0.0
365,361,15,26,12,1,кабинетом,ru,9,2.0
366,362,15,27,12,0,министров,ru,9,2.0


In [4]:
db = DataBundle()
text_df = pd.Series(text_result).to_frame('word_n')

In [5]:
vdf = pd.Series(vocab).to_frame().reset_index()
vdf.columns=['word','word_n']
vdf = vdf.merge(text_df.groupby('word_n').size().to_frame('cnt'), left_on='word_n', right_index=True)
vdf.sort_values('cnt', ascending = False)

,word,word_n,cnt
46,в,46,21844
3,и,3,10295
7,на,7,9007
199,<defis>,199,8746
26,что,26,5773
...,...,...,...
36011,фиксируют,36011,1
8534,паспортами,8534,1
36009,забитый,36009,1
36008,перекресток,36008,1


In [6]:
from jordan_lib.preprocessing import produce_ndf

ndf_0 = produce_ndf(vdf.loc[~vdf.word.str.startswith('<')])

  0%|          | 0/60704 [00:00<?, ?it/s]

  0%|          | 0/60704 [00:00<?, ?it/s]

In [7]:
from tg.grammar_ru.common import sample_table
ndf_0['parse'] = ndf_0.word+'⟶'+ndf_0.nf
sample_table(ndf_0, 'method', 'parse', 20)

,popularity,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
method,,,,,,,,,,,,,,,,,,,,,
DictionaryAnalyzer,112525,аббатстве⟶аббатство,аббревиатурой⟶аббревиатура,абдалла⟶абдалла,абдал⟶абдал,абдельбасет⟶абдельбасет,абдель⟶абдель,абдулатипов⟶абдулатипов,абдуллаев⟶абдуллаев,абдулла⟶абдулла,...,абдул⟶абдул,абиджана⟶абиджан,абиджане⟶абиджан,абитуриентам⟶абитуриент,абитуриенты⟶абитуриент,абитуриент⟶абитуриент,абонентами⟶абонент,абоненту⟶абонент,абонент⟶абонент,аборигенам⟶абориген
FakeDictionary-KnownSuffixAnalyzer,17641,абади⟶абади,абади⟶абадить,абади⟶абасть,аббассьяна⟶аббассьяна,аббаттобада⟶аббаттобад,абботтабадом⟶абботтабад,абдольреза⟶абдольрез,абдольреза⟶абдольрёза,абдольреза⟶абдольрёзый,...,абзаково⟶абзаковый,абзалова⟶абзаловый,абиев⟶абиев,абугаушу⟶абугауша,абугаушу⟶абугаушить,абхазско⟶абхазский,авакова⟶аваковый,авен⟶авный,авен⟶авня,аверелл⟶аверелла
DictionaryAnalyzer-UnknownPrefixAnalyzer,9195,абботабад⟶абботабад,абделатифа⟶абделатифа,абдуррахман⟶абдуррахман,абзалова⟶абзалов,авакова⟶авакова,авен⟶авена,авен⟶авено,авлаки⟶авлака,адагамов⟶адагамов,...,азавада⟶азавада,азоза⟶азоз,акар⟶акара,аквила⟶аквил,аквила⟶аквить,аквиле⟶аквил,аквиле⟶аквиль,акдага⟶акдага,акуила⟶акуил,аланью⟶аланья
FakeDictionary-KnownSuffixAnalyzer-KnownPrefixAnalyzer,830,автостат⟶автостатый,двухгосударственному⟶двухгосударственному,двухкилометровой⟶двухкилометровыть,двухпалубник⟶двухпалубник,двухпартийную⟶двухпартийный,дебоширку⟶дебоширка,дежурова⟶дежуровый,дежуров⟶дежур,дезинформирования⟶дезинформирование,...,деймон⟶деймон,дейнеко⟶дейнёкий,дельсаль⟶дельсалить,демократически⟶демократически,демонизацией⟶демонизация,деонтеем⟶деонтей,деонтею⟶деонтей,депозитарно⟶депозитарно,депозитарно⟶депозитарный,дерипаска⟶дерипаска
DictionaryAnalyzer-KnownPrefixAnalyzer,676,авиагруппировка⟶авиагруппировка,авианалета⟶авианалёт,авиапарку⟶авиапарк,авиапроисшествий⟶авиапроисшествие,автогола⟶автогол,автогола⟶автоголый,автогонкой⟶автогонка,автокомпонентов⟶автокомпонент,автоконцернам⟶автоконцерн,...,автоконцерном⟶автоконцерн,автокредитов⟶автокредит,автокредиту⟶автокредит,автопутешествий⟶автопутешествие,авторынка⟶авторынок,автосреда⟶автосреда,автотора⟶автотор,автотора⟶автотора,автотор⟶автотора,автоэкспедиция⟶автоэкспедиция
DictionaryAnalyzer-UnknownPrefixAnalyzer-KnownPrefixAnalyzer,427,автостат⟶автостат,автофрамос⟶автофрамос,артешина⟶артешина,делимханов⟶делимхановый,дельсаль⟶дельсалить,демехина⟶демехина,денбер⟶денбер,деонтею⟶деонтея,депозитарно⟶депозитарный,...,дерипаске⟶дерипаск,дерипаску⟶дерипаск,десинов⟶десиновый,десятикратному⟶десятикратное,детлева⟶детлев,детлева⟶детлева,детлева⟶детлевый,детлева⟶детлёва,дискаунтера⟶дискаунтёра,добкина⟶добкина
UnknAnalyzer,330,абд⟶абд,абэ⟶абэ,авб⟶авб,агс⟶агс,аеб⟶аеб,азс⟶азс,акг⟶акг,ак⟶ак,апл⟶апл,...,аро⟶аро,атр⟶атр,аун⟶аун,афк⟶афк,аш⟶аш,бдк⟶бдк,бе⟶бе,би⟶би,бкз⟶бкз,бла⟶бла
UnknAnalyzer-KnownPrefixAnalyzer,20,артени⟶артени,гоморры⟶гоморры,дебби⟶дебби,деппа⟶деппа,доган⟶доган,завос⟶завос,задой⟶задой,заема⟶заема,заеме⟶заеме,...,невки⟶невки,нежад⟶нежад,перейра⟶перейра,перейры⟶перейры,полухов⟶полухов,прадеш⟶прадеш,прадип⟶прадип,прайму⟶прайму,предова⟶предова,тележко⟶тележко
DictionaryAnalyzer-UnknownPrefixAnalyzer-KnownPrefixAnalyzer-KnownPrefixAnalyzer,17,незалежности⟶незалежность,интернетизации⟶интернетизации,,,,,,,,...,,,,,,,,,,


In [8]:
nf_0 = pd.DataFrame([
    dict(word='<unk>', nf='<unk>', nf_rule='preset'),
    dict(word='<defis>', nf='<defis>', nf_rule='preset'),
])
ndf_1 = ndf_0.loc[~ndf_0.word.isin(nf_0.word)]

# Reliable pymorphy analysis

In [9]:
def add_scoring(df):
    df = df.drop_duplicates(['word','nf'])
    df = df.merge(df.groupby('word').size().to_frame('alternatives'), left_on='word', right_index=True)
    df = df.merge(df.groupby('nf').cnt.sum().to_frame('nf_popularity'), left_on='nf', right_index=True)
    df = df.feed(fluq.add_ordering_column('word',('nf_popularity', False), 'nf_pop_order'))
    df = df.feed(fluq.add_ordering_column('word',('scores', False), 'nf_scores_order'))
    return df

reliable = [
    'DictionaryAnalyzer',
    'DictionaryAnalyzer-KnownPrefixAnalyzer'
]

ndf_rel = ndf_1.loc[ndf_1.method.isin(reliable)]
ndf_rel = add_scoring(ndf_rel)
nf_1 = pd.concat([nf_0, ndf_rel.loc[ndf_rel.nf_pop_order==0][['word','nf', 'nf_scores_order']].assign(nf_rule='pymorphy')])
nf_1[nf_1.word.isin(nf_1[nf_1['nf_scores_order'] > 0].word)].head(50)
nf_1.sample(10)

,word,nf,nf_rule,nf_scores_order
131265,экспоната,экспонат,pymorphy,0.0
18892,фильма,фильм,pymorphy,0.0
65355,апостолы,апостол,pymorphy,0.0
23840,напряжение,напряжение,pymorphy,0.0
129701,аккредитивов,аккредитив,pymorphy,0.0
49532,легендарным,легендарный,pymorphy,0.0
56180,хартии,хартия,pymorphy,0.0
18893,звучит,звучать,pymorphy,0.0
138593,заселили,заселить,pymorphy,0.0
33604,находящееся,находиться,pymorphy,0.0


# Stemming of non-vocab words

In [10]:
ndf_2 = ndf_1.loc[~ndf_1.word.isin(nf_1.word)]

In [11]:
mdf = ndf_2.drop_duplicates(['word','stem'])

sdf = mdf.feed(fluq.add_ordering_column('stem',('cnt',False),'word_in_stem_order'))
sdf = sdf.loc[sdf.word_in_stem_order==0]
sdf = sdf.drop('nf',axis=1).rename(columns=dict(word='nf')).set_index('stem')[['nf']]

rdf = mdf[['word','stem', 'cnt']].merge(sdf, left_on='stem', right_index=True)
rdf = rdf.merge(rdf.groupby('nf').cnt.sum().to_frame('nf_cnt'), left_on='nf', right_index=True)
rdf['log_nf_cnt'] = np.log(rdf.nf_cnt)/np.log(2)
rdf.log_nf_cnt = rdf.log_nf_cnt.astype(int)

sample_table(rdf.drop_duplicates(['nf']).sample(200), 'log_nf_cnt', 'nf', 30).sort_index().transpose()

log_nf_cnt,0,1,2,3,4,6
popularity,126,51,12,7,3,1
0,анадолу,абхазо,бермейном,ассанж,месси,нью
1,аннане,акино,губоп,дипотношений,русала,
2,аудирован,атинской,кдк,корреа,чуров,
3,баматгиреевичу,баскаева,линшиц,минрегионразвития,,
4,башахшехиру,бейкерсфилд,мелилья,поветкина,,
5,брегнец,боко,твц,сорочкина,,
6,бубнива,велма,хашеми,юсуфзай,,
7,бэрримор,воопик,хейли,,,
8,вилс,гудаутской,цдм,,,


In [12]:
nf_2 = pd.concat([nf_1, rdf.loc[rdf.log_nf_cnt>1][['word','nf']].assign(nf_rule='stem')])
ndf_3 = ndf_2.loc[~ndf_2.word.isin(nf_2.word)]

## Residuals

In [13]:
nf_3 = pd.concat([nf_2, ndf_3.drop_duplicates('word')[['word']].assign(nf='<unk>', nf_rule='residuals')])
nf_3

,word,nf,nf_rule,nf_scores_order
0,<unk>,<unk>,preset,NaN
1,<defis>,<defis>,preset,NaN
212,а,а,pymorphy,0.0
116451,аарона,аарон,pymorphy,0.0
106255,абакаров,абакар,pymorphy,0.0
...,...,...,...,...
141543,пелем,<unk>,residuals,NaN
141545,фрейтс,<unk>,residuals,NaN
141594,никсона,<unk>,residuals,NaN
141618,белыом,<unk>,residuals,NaN


# Combining everything

In [14]:
nf = nf_3.reset_index(drop=True)
assert len(nf.groupby('word').size().feed(lambda z: z.loc[z>1]))==0
nf_voc = nf.nf.drop_duplicates().to_frame('nf').reset_index(drop=True).reset_index(drop=False)
nf_voc.columns = ['nf_n','nf']
nf = nf.merge(nf_voc.set_index('nf').nf_n, left_on='nf', right_index=True, how='left')
nf_voc = nf_voc.set_index('nf_n')
assert not nf.nf_n.isnull().any()
print(nf_voc.shape[0])
nf

20476


,word,nf,nf_rule,nf_scores_order,nf_n
0,<unk>,<unk>,preset,NaN,0
1,<defis>,<defis>,preset,NaN,1
2,а,а,pymorphy,0.0,2
3,аарона,аарон,pymorphy,0.0,3
4,абакаров,абакар,pymorphy,0.0,4
...,...,...,...,...,...
60701,пелем,<unk>,residuals,NaN,0
60702,фрейтс,<unk>,residuals,NaN,0
60703,никсона,<unk>,residuals,NaN,0
60704,белыом,<unk>,residuals,NaN,0


In [15]:
words = vdf.merge(nf.set_index('word')[['nf_n','nf_rule']], left_on='word', right_index=True, how='left')
assert not words.nf_n.isnull().any()
words = words.set_index('word_n')
words

,word,cnt,nf_n,nf_rule
word_n,,,,
0,компромисс,3,6801,pymorphy
1,между,409,8175,pymorphy
2,депутатами,12,3906,pymorphy
3,и,10295,5519,pymorphy
4,правительством,39,13006,pymorphy
...,...,...,...,...
60700,продержится,1,13706,pymorphy
60701,четырехлетнего,2,19097,pymorphy
60702,квиннипэкского,1,0,residuals


In [16]:
nf_voc_full = nf_voc.merge(words.groupby('nf_n').cnt.sum().to_frame('cnt'), left_index=True, right_index=True)

In [17]:
bundle = DataBundle(text=text_df, words=words, nfs=nf_voc_full)
bundle.save_as_zip(rucoco_dir/'bundle.zip')